<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script preprocesses Russian-language text:
Splits the text into sentences and words (removing hashtags)

*   Removes characters outside the Russian alphabet
*   Excludes stop words (built-in + custom lists from GitHub)
*   Filters out emoji, URLs and single-letter tokens

*   Normalizes words via PyMorphy3 (reduces to the initial form)
*   Optionally adds punctuation marks (dots)
*   Returns a cleaned and lemmatized string.





Этот скрипт выполняет предобработку русскоязычного текста:
Разбивает текст на предложения и слова (удаляя хэштеги)

*    Удаляет символы вне русского алфавита

*   Исключает стоп-слова (встроенные + кастомные списки с GitHub)
*  Отфильтровывает эмодзи, URL и однобуквенные токены
*  Нормализует слова через PyMorphy3 (приводит к начальной форме)
*  Опционально добавляет знаки препинания (точки)
*  Возвращает очищенную и лемматизированную строку.




In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import pymorphy3
import requests
import csv
import emoji
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)


stop_words = stopwords.words("russian")
stop_words += requests.get('https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/src/vk/nlp/RussianStopWords.txt').text.split('\n')
stop_words += requests.get('https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/src/vk/nlp/stopwords-ru.txt').text.split()

alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя-.')
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [ ]:
def process_text(text, points=False):
    sentences = sent_tokenize(text)
    processed_sentences = []

    for sentence in sentences:
        processed_parts = []
        tokenize_sent = [t for t in word_tokenize(sentence) if not t.startswith('#')]
        for w in word_tokenize(tokenize_sent):
            if (len(w) == 1 or
                ":" in emoji.demojize(w) or
                not set(w.lower()).issubset(alphabet) or
                not w[0:2].lower().isalpha() or
                '\\' in w or
                '/' in w):
                continue

            res = morph.parse(w.lower())[0].normal_form
            if res and (res not in stop_words):
                processed_parts.append(res)

        if points and processed_parts:
            processed_parts[-1] += "."

        processed_sentences.append(" ".join(processed_parts))

    return " ".join(processed_sentences)